In [1]:
from IPython.display import clear_output  #use to clear output
from motor import XM430 , XL320

import traceback

import numpy as np
import pandas as pd
import matplotlib as mpl

import cv2
from cv2 import aruco
import pickle

import time
import keyboard

import math


defaultLocation=[XL320.dxl_home_position +20,XL320.dxl_home_position +80,XL320.dxl_home_position -20,XL320.dxl_home_position -80]

In [2]:
class finished(Exception):
    """Base class for other exceptions"""
    pass

#Generator to serve moving positions for the XL320 motors step number of times starting from start location
def moveXL320(steps,base,digit):
    
    change=0
    scale=0
    dxl_home_position_manipulation=XL320.dxl_home_position +(base)
    dx2_home_position_manipulation=XL320.dxl_home_position +(digit +scale)
    dx3_home_position_manipulation=XL320.dxl_home_position -(base)
    dx4_home_position_manipulation=XL320.dxl_home_position -(digit +scale)
    
    print(f'base values: {[dxl_home_position_manipulation,dx2_home_position_manipulation,dx3_home_position_manipulation,dx4_home_position_manipulation]}')
    print(f'base : {base} , digit:{digit +scale}')
    yield [dxl_home_position_manipulation,dx2_home_position_manipulation,dx3_home_position_manipulation,dx4_home_position_manipulation]
    #Move in first direction
    for i in range(1,steps):
        dxl_goal_position_manipulation=dxl_home_position_manipulation - 5* i
        dx3_goal_position_manipulation=dx3_home_position_manipulation - 5* i
        change=yield [dxl_goal_position_manipulation,dx2_home_position_manipulation,dx3_goal_position_manipulation,dx4_home_position_manipulation]
        if change:
            break
    yield True #Return true at endo f first round
    #Move in second direction
    for i in range(1,steps):
        dxl_goal_position_manipulation=dxl_home_position_manipulation + 5* i
        dx3_goal_position_manipulation=dx3_home_position_manipulation + 5* i
        change=yield [dxl_goal_position_manipulation,dx2_home_position_manipulation,dx3_goal_position_manipulation,dx4_home_position_manipulation]
        if change:
            break
    yield False #Return false at end of second round


#Generator that supplies base positions for the xl320. These values are calcualted before due to fully actuated nature
def increaseBasePositionXL320():
    base_positions=[13,18,22,26,29,33,35,38,41,43,46,48,50,52,54,56,58,60,62,63,65,67,68,70,71,73,74,76,77,78,80,81,82,84,85,86,87]
    digit_positions=[102,115,126,135,144,152,160,167,173,179,186,191,197,202,208,213,218,222,227,232,236,241,245,249,253,257,261,265,269,273,277,281,284,288,292,295,299]
    steps=[10,10,10,10,15,15,15,15,15,25,25,25,25,25,30,30,30,30,30,30,30,30,30,30,30,30,30,30,25,25,25,25,25,25,25,25,25]
    count=0
    
    while count < len(base_positions):
        yield steps[count],base_positions[count],digit_positions[count]
        #yield 3,base_positions[count],digit_positions[count]
        print(f'count: {count}')
        count +=1
    return

#Return the gripper to the center
def retrunHomeXL320(motors,base,digit,values):
    
    dxl_goal_position_manipulation=XL320.dxl_home_position +(base)
    dx2_goal_position_manipulation=XL320.dxl_home_position +(digit)
    dx3_goal_position_manipulation=XL320.dxl_home_position -(base)
    dx4_goal_position_manipulation=XL320.dxl_home_position -(digit)
    basePositions=[dxl_goal_position_manipulation,dx2_goal_position_manipulation,dx3_goal_position_manipulation,dx4_goal_position_manipulation]
    currentPosition=motors.getAllPositions()
    distance=currentPosition[0]-basePositions[0]
    positions=currentPosition
    #print("distance: " + str(distance))
    if abs(distance) <5: # if less than 5 away then just move to base position
        motors.writeGroupPosition(basePositions)

    else:
        num=10
        #calculae number of movements from current position
        value=(abs(distance))//num
        #print("value is:" + str(value))

        for j in range(1,value):
            #Check the direction needed
            if distance <0:
                positions[0]=positions[0]+num-2
                positions[2]=positions[2]+num
            else:
                positions[0]=positions[0]-num
                positions[2]=positions[2]-num+2
            
            motors.writeGroupPosition(positions)
            time.sleep(.25)
            
        #Move to the base position

        motors.writeGroupPosition(basePositions)
    if len(values) <5:
        movevalues=values[::-1]
    else:   
        movevalues=values[::-4]
        movevalues.append(values[0])
    #print(f"return_home:{movevalues}")
    for base,digit in movevalues:
        motors.writeGroupPosition([XL320.dxl_home_position +base,XL320.dxl_home_position +digit,XL320.dxl_home_position -base,XL320.dxl_home_position -digit])
        time.sleep(0.25)
    
#Reset the grasp on the object
def resetObject(motors,values):
    motors.writeGroupPosition(defaultLocation)
    time.sleep(1)
    motors.openGripper()
    time.sleep(1)
    motors.raiseObject()
    time.sleep(2.5)
    motors.writeGroupPosition([XL320.dxl_home_position +13,XL320.dxl_home_position +100,XL320.dxl_home_position -13,XL320.dxl_home_position -100])
    time.sleep(1)
    motors.lowerObject()
    if len(values) <5:
        movevalues=values
    else:   
        movevalues=values[:6]
        movevalues+=values[6::4]
        movevalues.append(values[-1])
    for base,digit in movevalues:
        motors.writeGroupPosition([XL320.dxl_home_position +base,XL320.dxl_home_position +digit,XL320.dxl_home_position -base,XL320.dxl_home_position -digit])
        time.sleep(0.25)
   



# Checks if a matrix is a valid rotation matrix.
# code from https://learnopencv.com/rotation-matrix-to-euler-angles/
def isRotationMatrix(R) :
    Rt = np.transpose(R)
    shouldBeIdentity = np.dot(Rt, R)
    I = np.identity(3, dtype = R.dtype)
    n = np.linalg.norm(I - shouldBeIdentity)
    return n < 1e-6

# Calculates rotation matrix to euler angles
# code from https://learnopencv.com/rotation-matrix-to-euler-angles/
def rotationMatrixToEulerAngles(R) :

    assert(isRotationMatrix(R))

    sy = math.sqrt(R[0,0] * R[0,0] +  R[1,0] * R[1,0])

    singular = sy < 1e-6

    if  not singular :
        x = math.atan2(R[2,1] , R[2,2])
        y = math.atan2(-R[2,0], sy)
        z = math.atan2(R[1,0], R[0,0])
    else :
        x = math.atan2(-R[1,2], R[1,1])
        y = math.atan2(-R[2,0], sy)
        z = 0

    return np.array([x, y, z])
    

In [ ]:
#Make the colums and datafram for pandas
column_names=["motor 1", "motor 2", "motor 3", "motor 4", "X axis","y axis", "z Axis", "x rotation", "y rotation", "z rotation","movement Round"]
data=pd.DataFrame(columns = column_names)



shape="Square" # input("what is the shape")
tips=input("what are the finger tips")


#Base Vales
baseLocation=[XL320.dxl_home_position +13,XL320.dxl_home_position +80,XL320.dxl_home_position -13,XL320.dxl_home_position -80]



camera=1                #Camera selection variable
counter=0               #Counter to add a movement delay without interrupting camera
markerSize=2.5          #Marker size in cm

xl320=XL320("COM3")     #Create a xl320 object on COM

# Open port
result=xl320.initMotorControl()
if result != 0:
    print("error starting motors")
    quit()

#Set basic information for the motors
xl320.setPID(50,0,0)
xl320.setGoalVelocity(512)
xl320.enableTorque()
xl320.turnOnLED(7)

try:
    #Use the calibration of the camera. See aruco_calibration.ipynb for calibration details
    with open(f"truc{camera}.pckl", "rb") as f:
        calibrationParameters = pickle.load(f)

    cameraMatrix=calibrationParameters[1]
    distCoeffs=calibrationParameters[2]
    #print(cameraMatrix)
    #print(distCoeffs)
except:
    print("need to calibrate")


aruco_dictionary = aruco.Dictionary_get(cv2.aruco.DICT_4X4_50)

# Create parameters to be used when detecting markers:
parameters = aruco.DetectorParameters_create()

#Open CV capture and set frame dimensions
cap = cv2.VideoCapture(camera,cv2.CAP_DSHOW)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)



for i in range(3):
    Movement_round =0 #variable for which itteration on
    values=[]               #Array to hold the values of the base motor positions
    #Get a generator for the base locations of the moment pattern
    basePosition=increaseBasePositionXL320()
    steps,base,digit=next(basePosition)
    values.append((base,digit))

    resetObject(xl320,values)

    #Create a mover generator
    mover=moveXL320(steps,base,digit)
    nextPosition=0
    try:

        while True:
            # Capture frame by frame from the video capture object 'capture':
            ret, frame = cap.read()
            # We convert the frame to grayscale:
            gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

            # lists of ids and the corners beloning to each id# We call the function 'cv2.aruco.detectMarkers()'
            corners, ids, rejectedImgPoints = aruco.detectMarkers(gray_frame, aruco_dictionary, parameters=parameters)
            # Draw detected markers:
            frame = aruco.drawDetectedMarkers(image=frame, corners=corners, borderColor=(0, 255, 0))

            # Draw rejected markers:
            #frame = aruco.drawDetectedMarkers(image=frame, corners=rejectedImgPoints, borderColor=(0, 0, 255))

            

            # Display the resulting frame
            cv2.imshow('frame', frame)
            
            #Check if the motor is moving or during counter
            if not xl320.isMoving()and counter ==0:

                    #Look for Arucos. If nothing then take a new frame
                if ids is not None:
                    # Get the location of the corners of the marker and calculate the center
                    rvecs, tvecs, _ = aruco.estimatePoseSingleMarkers(corners, markerSize, cameraMatrix, distCoeffs)
                    for rvec, tvec in zip(rvecs, tvecs):
                        aruco.drawAxis(frame, cameraMatrix, distCoeffs, rvec, tvec, 1)

                    centervector=tvecs[0][0]

                    rotation=cv2.Rodrigues(rvecs)
                    rotationvector=rotationMatrixToEulerAngles(rotation[0])

                else:
                    #Else do not record anything
                    continue

                #Get the current motor positions and other information and form that into an array
                # Create a pandas data frame and append to the full data frame
                currentPosition=xl320.getAllPositions()
                currentPosition.extend(centervector)
                currentPosition.extend(rotationvector)
                currentPosition.extend([Movement_round])
                newdata=pd.Series(currentPosition,index=data.columns)
                data=data.append(newdata, ignore_index=True)


                #Get the next positon value to move to
                positions=next(mover)

                
                if positions is True: #Generator will return True at turn around point
                
                #Return to the center position
                    retrunHomeXL320(xl320,base,digit,values)
                    
                    #Reset the object for the next pass

                    time.sleep(1)
                    resetObject(xl320,values)

                    positions=next(mover)
                    xl320.writeGroupPosition(positions)
                    time.sleep(1)


                elif positions is False: #Generator will return False at the end 
                    #Reset the position to the home location
                    retrunHomeXL320(xl320,base,digit,values)
                    time.sleep(1)

                    resetObject(xl320,values)

                    Movement_round+=1   #Increase movement round as we will move to new base position
                    steps,base,digit = next(basePosition)
                    values.append((base,digit))
                    mover=moveXL320(steps,base,digit)

                else:
                    xl320.writeGroupPosition(positions)

                counter +=1
                #Counter used to provide delay but not interfer with camera
            elif counter>=5:
                counter =0
            else:
                counter +=1
            
            
            #Break if q is pressed while the camera is working
            if cv2.waitKey(10) & 0xFF == ord('q'):
                break
            #Break if ~ sign
            if keyboard.is_pressed("`"):
                break


    except StopIteration:
        print("end of positions")

    except:
        print(traceback.format_exc())
        # Disable Dynamixel Torque
        print("exception")


  #Save the data
    finally:
        print("write dataS")
        filename=f"{shape} {tips} {i}"
        data.to_csv("..\data\\fully " +filename+".csv")
        data=data[0:0]


#Release the object and close the camera and motor comunications
xl320.openGripper()
time.sleep(1)
xl320.shutdown()
# Release everything:
cap.release()
cv2.destroyAllWindows()
